In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../1.1 Data Ingestion/speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs = text_splitter.split_documents(documents)


In [4]:
embeddings = OllamaEmbeddings(model = "embeddinggemma")
db = FAISS.from_documents(docs,embeddings)
db

## Query vector database

In [6]:
query = "What does the speaker believe is the main reason the united states should enter the war?"
query = "How does the speaker describe the desired outcome of the war?"
results = db.similarity_search(query)
results[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

## As a retriever
We can convert the db into a retriever class. This allows us to easily use it in another Langchain methods.

In [8]:
retriever = db.as_retriever()
results = retriever.invoke(query)
results[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

## Similarity search with score

In [9]:
results_and_score = db.similarity_search_with_score(query)
results_and_score

[(Document(id='e0d94105-3c2c-4057-ac72-ed5721e2c64d', metadata={'source': '../1.1 Data Ingestion/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  0.8509247),
 (Document(id='91aadb34-6e03-4cdd-ae99-0f5c0b3542cf', metadata={'source': '../1.1 Data Ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when

## Search with vectors

In [10]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-0.13239454,
 0.048678778,
 0.021848693,
 0.010480481,
 -0.01058842,
 0.032281823,
 -0.0626545,
 0.015993167,
 0.02289075,
 -0.046156213,
 0.056177445,
 -0.028185528,
 0.055344112,
 0.009225912,
 0.089604914,
 0.02703648,
 0.023506286,
 -0.0041586016,
 -0.088895306,
 0.0008847334,
 0.030605504,
 -0.063256286,
 0.00015095684,
 -0.017617466,
 -0.0019277566,
 0.052277405,
 0.0027485031,
 -0.0097619565,
 -0.04387633,
 -0.025086708,
 0.03859241,
 0.026340602,
 0.06239334,
 -0.009090448,
 -0.016020307,
 0.021626966,
 0.012199279,
 -0.062077574,
 0.005565876,
 -0.02197112,
 -0.054524086,
 0.08233274,
 -0.010010842,
 -0.043429088,
 0.0050061718,
 -0.03710606,
 -0.028214896,
 0.007558602,
 -0.03674906,
 0.034731813,
 -0.033394597,
 0.012506328,
 -0.02548397,
 0.0018998382,
 -0.040952228,
 -0.053956315,
 0.0043220124,
 0.024597697,
 -0.035143208,
 -0.027271733,
 -0.068909444,
 -0.047659744,
 -0.01553081,
 0.028865047,
 0.009971624,
 0.015596521,
 -0.027196774,
 -0.009593318,
 0.024009516,
 0.26

In [11]:
result_vector = db.similarity_search_by_vector(embedding_vector)
result_vector

[Document(id='e0d94105-3c2c-4057-ac72-ed5721e2c64d', metadata={'source': '../1.1 Data Ingestion/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='91aadb34-6e03-4cdd-ae99-0f5c0b3542cf', metadata={'source': '../1.1 Data Ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights ha

## Saving and loading

In [12]:
db.save_local("faiss_index")

In [15]:
new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
results = new_db.similarity_search(query)
results

[Document(id='e0d94105-3c2c-4057-ac72-ed5721e2c64d', metadata={'source': '../1.1 Data Ingestion/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='91aadb34-6e03-4cdd-ae99-0f5c0b3542cf', metadata={'source': '../1.1 Data Ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights ha